# CS 534 - Machine Learning 
## PS3
### Mostofa Najmus Sakib
### St. Id: 114102108

##Question1(15pts) (Scikit Allowed)To  classify  iris  dataset  (Iris-Versicolor  vs.  others)  in  the  best  way,  you  have  to  create  an  algorithm  able  to determine (with the k-fold cross validation) what is the best space transformation (among rbf_kernel, polynomial features,  and  polynomial  kernel)and  its  hyperparameters.  Each  transformation  has  its  own  parameters rbf_kernel->gamma,  polynomial-features->  degree,  polynomial_kernel ->(gamma,  degree).  The  performance mustbe tested of the entire algorithm with another K-fold cross-validation (please use then trainset, validationset, and testset).

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import polynomial_kernel
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from warnings import filterwarnings
filterwarnings('ignore')

Reading iris-dataset(Iris-Versicolor vs. others):

In [3]:
#1 means Iris-versicolor, 0 means other
def getBinaryClass(data):
    result = []
    
    for i in range(0, len(data)):
        if data.iloc[i]['class'] == "Iris-versicolor":
            result.append(1)
        else:
            result.append(0)
    return result

In [4]:
iris_data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',header=None)
iris_data.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class']
iris_data['binary_class'] = getBinaryClass(iris_data)
#1 means Iris-versicolor, 0 means other
iris_data = iris_data.drop(columns=['class'])
iris_data.head()

,sepal length,sepal width,petal length,petal width,binary_class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
columns = ['sepal length', 'sepal width', 'petal length', 'petal width']
iris_data_X = pd.DataFrame(iris_data, columns=columns)
iris_data_X.head()

,sepal length,sepal width,petal length,petal width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [6]:
iris_data_Y = iris_data['binary_class']
iris_data_Y.head()

0    0
1    0
2    0
3    0
4    0
Name: binary_class, dtype: int64

In [7]:
skf = StratifiedKFold(n_splits=10,random_state=10,shuffle=True)
#totalDataframe contains the result for all our possible iteration, we will use this dataframe in the end
#to get the final best result
totalDataFrame = pd.DataFrame(columns = ['kernel', 'degree', 'gamma', 'accuracy'])

In [8]:
def rbfKernel(dataX, dataY, gammas):
    dataY = dataY.reset_index()
    dataY = dataY['binary_class']
    global totalDataFrame
    fold = 0
    for gamma in gammas:
        rbf_dataX=rbf_kernel(dataX, gamma=gamma)
        predictor = LogisticRegression(random_state=0).fit(rbf_dataX,dataY)
        accuracy = 0
        for train_index, validation_index in skf.split(rbf_dataX, dataY):
            fold = fold + 1
            X_train=rbf_dataX[train_index]
            X_val=rbf_dataX[validation_index ]
            y_train=dataY[train_index]
            y_val=dataY[validation_index ]
            y_pred=predictor.predict(X_val)
            accuracy +=  accuracy_score(y_val, y_pred)
        acc = accuracy/10
        kernel = "rbf_kernel"
        degree = "None"
        data = [(kernel, degree, gamma, acc)]
        temp = pd.DataFrame(data, columns = ['kernel', 'degree', 'gamma', 'accuracy'])
        totalDataFrame = pd.concat([totalDataFrame, temp])

In [9]:
def PolyFeatureKernel(dataX, dataY, degrees):
    dataY = dataY.reset_index()
    dataY = dataY['binary_class']
    global totalDataFrame
    fold = 0
    
    for degree in degrees:
        poly = PolynomialFeatures(degree)
        polyF_dataX = poly.fit_transform(dataX)
        
        predictor = LogisticRegression(random_state=0).fit(polyF_dataX,dataY)
        accuracy = 0
        for train_index, validation_index in skf.split(polyF_dataX, dataY):
            fold+=1
            X_train=polyF_dataX[train_index]
            X_val=polyF_dataX[validation_index ]
            y_train=dataY[train_index]
            y_val=dataY[validation_index ]
            y_pred=predictor.predict(X_val)
            accuracy +=  accuracy_score(y_val, y_pred)
        acc = accuracy/10
        kernel = "polynomial_features"
        gamma = "None"
        data = [(kernel, degree, gamma, acc)]
        temp = pd.DataFrame(data, columns = ['kernel', 'degree', 'gamma', 'accuracy'])
        totalDataFrame = pd.concat([totalDataFrame, temp])



In [10]:
def PolyKernel(dataX, dataY, degrees, gammas):
    dataY = dataY.reset_index()
    dataY = dataY['binary_class']
    global totalDataFrame
    fold = 0
    norm='l2'
    normalized_poly_dataX = preprocessing.normalize(dataX, norm=norm)
    
    for degree in degrees:
        
        for gamma in gammas:
            accuracy = 0
            poly_dataX=polynomial_kernel(normalized_poly_dataX, degree= degree, gamma = gamma)
            
            predictor = LogisticRegression(random_state=0).fit(poly_dataX,dataY)
             
            for train_index, validation_index in skf.split(poly_dataX, dataY):
                fold = fold + 1
                X_train=poly_dataX[train_index]
                X_val=poly_dataX[validation_index ]
                y_train=dataY[train_index]
                y_val=dataY[validation_index ]
                y_pred=predictor.predict(X_val)
                accuracy +=  accuracy_score(y_val, y_pred)
            acc = accuracy/10
            #print(acc)
            kernel = "polynomial_kernel"
            data = [(kernel, degree, gamma, acc)]
            temp = pd.DataFrame(data, columns = ['kernel', 'degree', 'gamma', 'accuracy'])
            totalDataFrame = pd.concat([totalDataFrame, temp])


In [11]:
sampleGammas = [0.001, 0.01, 0.1, 1]
sampleDegrees = [1, 2, 3]


def ExecuteBestTransformationLookup(dataX, dataY, degreeList, gammaList):
    outerFold = 0
    for train_index, test_index in skf.split(iris_data_X, iris_data_Y):
        outerFold = outerFold + 1
        X_train=dataX.loc[train_index]
        X_test=dataX.loc[test_index]
        y_train=dataY.loc[train_index]
        y_test=dataY.loc[test_index]
        
        
        scaler = StandardScaler()
        XS_train=scaler.fit_transform(X_train) 
        XS_test=scaler.fit_transform(X_test)
        PolyFeatureKernel(XS_train, y_train, degreeList)
        rbfKernel(XS_train, y_train, gammaList)
        PolyKernel(XS_train, y_train, degreeList, gammaList)

        
ExecuteBestTransformationLookup(iris_data_X, iris_data_Y, sampleDegrees, sampleGammas)

In [12]:
totalDataFrame = totalDataFrame.reset_index(drop=True)
totalDataFrame

,kernel,degree,gamma,accuracy
0,polynomial_features,1,None,0.755495
1,polynomial_features,2,None,0.985165
2,polynomial_features,3,None,0.985165
3,rbf_kernel,None,0.001,0.667582
4,rbf_kernel,None,0.01,0.837912
...,...,...,...,...
185,polynomial_kernel,2,1,0.903297
186,polynomial_kernel,3,0.001,0.667582
187,polynomial_kernel,3,0.01,0.667582
188,polynomial_kernel,3,0.1,0.821978


the "totalDataframe" contains the accuracy for all our iteration of the outer kfold cross validation loop. For example, it contains 10 rows for rbf_kernel with gamma 0.1. let's verify that it is true.

In [13]:
totalDataFrame[(totalDataFrame['kernel'] == "rbf_kernel") & (totalDataFrame['gamma'] == 0.1)]

,kernel,degree,gamma,accuracy
5,rbf_kernel,None,0.1,0.890659
24,rbf_kernel,None,0.1,0.896154
43,rbf_kernel,None,0.1,0.912088
62,rbf_kernel,None,0.1,0.910989
81,rbf_kernel,None,0.1,0.897802
100,rbf_kernel,None,0.1,0.896703
119,rbf_kernel,None,0.1,0.896703
138,rbf_kernel,None,0.1,0.903846
157,rbf_kernel,None,0.1,0.874176
176,rbf_kernel,None,0.1,0.882418


Therefore We need to write a function that will calculate the average accuracy for all our outer k-fold validation loop for all possible kernel, gamma, degree combination.

In [14]:
sampleGammas = [0.001, 0.01, 0.1, 1, "None"]
sampleDegrees = [1, 2, 3, "None"]
listOfKernels = ["rbf_kernel", "polynomial_features", "polynomial_kernel"]

def getAllPossibleAvgAccuracy(data):
    result = pd.DataFrame(columns = ['kernel', 'degree', 'gamma', 'accuracy'])
    
    for i in range(0, len(listOfKernels)):
        for j in range(0, len(sampleGammas)):
            for k in range(0, len(sampleDegrees)):
                temp = data[(data['kernel'] == listOfKernels[i]) & (data['gamma'] == sampleGammas[j]) & (data['degree'] == sampleDegrees[k])]
                avgAccuracy = temp['accuracy'].sum() / len(temp['accuracy'])
                
                dataRow = [(listOfKernels[i], sampleDegrees[k], sampleGammas[j], avgAccuracy)]
                tempDF = pd.DataFrame(dataRow, columns = ['kernel', 'degree', 'gamma', 'accuracy'])
                result = pd.concat([result, tempDF])
    result = result.dropna()
    return result


allPossibleAvgAccuracy = getAllPossibleAvgAccuracy(totalDataFrame)

In [15]:
allPossibleAvgAccuracy

,kernel,degree,gamma,accuracy
0,rbf_kernel,None,0.001,0.667582
0,rbf_kernel,None,0.01,0.842253
0,rbf_kernel,None,0.1,0.896154
0,rbf_kernel,None,1,0.974011
0,polynomial_features,1,None,0.731154
0,polynomial_features,2,None,0.983571
0,polynomial_features,3,None,0.983626
0,polynomial_kernel,1,0.001,0.667582
0,polynomial_kernel,2,0.001,0.667582
0,polynomial_kernel,3,0.001,0.667582


Now let's see which is our best accuracy scenario for our short example with limited gamma and degree values.

In [16]:
maxRes = allPossibleAvgAccuracy[allPossibleAvgAccuracy['accuracy'] == allPossibleAvgAccuracy['accuracy'].max()]
maxRes

,kernel,degree,gamma,accuracy
0,polynomial_features,3,None,0.983626


##Question2(10pts)Write the explicit constraints (without using any vectorial notation, as a summation of single variables multiplied by a constant + bias term) of the Support Vector Machine to correctly classifyiris dataset (Iris-Versicolor vs. others). Use5 points in Iris-Versicolor, 2 points for iris-setosa, and 3 points for iris Virginia.Please show the points you selected and after the constraints.

In [17]:
iris_data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',header=None)
iris_data.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class']

iris_data.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [18]:
iris_data['class'].value_counts()

Iris-virginica     50
Iris-setosa        50
Iris-versicolor    50
Name: class, dtype: int64

In [19]:
#selecting 5 points of Iris-versicolor
versicolor = iris_data[iris_data['class'] == 'Iris-versicolor']
versicolor = versicolor.head(5)

#selecting 2 points of Iris-setosa
setosa = iris_data[iris_data['class'] == 'Iris-setosa']
setosa = setosa.head(2)

#selecting 3 points of Iris-virginica
virginica = iris_data[iris_data['class'] == 'Iris-virginica']
virginica = virginica.head(3)

print('5 points for Iris-versicolor', versicolor)
print('2 points for Iris-setosa', setosa)
print('3 points for Iris-virginica', virginica)

5 points for Iris-versicolor     sepal length  sepal width  petal length  petal width            class
50           7.0          3.2           4.7          1.4  Iris-versicolor
51           6.4          3.2           4.5          1.5  Iris-versicolor
52           6.9          3.1           4.9          1.5  Iris-versicolor
53           5.5          2.3           4.0          1.3  Iris-versicolor
54           6.5          2.8           4.6          1.5  Iris-versicolor
2 points for Iris-setosa    sepal length  sepal width  petal length  petal width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
3 points for Iris-virginica      sepal length  sepal width  petal length  petal width           class
100           6.3          3.3           6.0          2.5  Iris-virginica
101           5.8          2.7           5.1          1.9  Iris-virginica
102           7.1          3.0           5.9 

In [20]:
versicolor

,sepal length,sepal width,petal length,petal width,class
50,7.0,3.2,4.7,1.4,Iris-versicolor
51,6.4,3.2,4.5,1.5,Iris-versicolor
52,6.9,3.1,4.9,1.5,Iris-versicolor
53,5.5,2.3,4.0,1.3,Iris-versicolor
54,6.5,2.8,4.6,1.5,Iris-versicolor


In [21]:
setosa

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa


In [22]:
virginica

,sepal length,sepal width,petal length,petal width,class
100,6.3,3.3,6.0,2.5,Iris-virginica
101,5.8,2.7,5.1,1.9,Iris-virginica
102,7.1,3.0,5.9,2.1,Iris-virginica


## Assume Iris-versicolor class is represented by -1 and others (i.e. Iris-setosa and Iris-virginica) are 1. 
## Now for Iris-versicolor constraints are: 
## w0 * 7.0 + w1 * 3.2 + w2 * 4.7 + w3 * 1.4 + b1 <= -1
## w0 * 6.4 + w1 * 3.2 + w2 * 4.5 + w3 * 1.5 + b1 <= -1
## w0 * 6.9 + w1 * 3.1 + w2 * 4.9 + w3 * 1.5 + b1 <= -1 
## w0 * 5.5 + w1 * 2.3 + w2 * 4.0 + w3 * 1.3 + b1 <= -1
## w0 * 6.5 + w1 * 2.8 + w2 * 4.6 + w3 * 1.5 + b1 <= -1 
## for others (i.e. Iris-setosa and Iris-virginica) constraints are:
## constraints for Iris-setosa:
## w0 * 5.1 + w1 * 3.5 + w2 * 1.4 + w3 * 0.2 + b2 >= 1
## w0 * 4.9 + w1 * 3.0 + w2 * 1.4 + w3 * 0.2 + b2 >=1 
## constraints for Iris-virginica: 
## w0 * 6.3 + w1 * 3.3 + w2 * 6.0 + w3 * 2.5 + b3 >= 1
## w0 * 5.8 + w1 * 2.7 + w2 * 5.1 + w3 * 1.9 + b3 >= 1
## w0 * 7.1 + w1 * 3.0 + w2 * 5.9 + w3 * 2.1 + b3 >= 1


##Extra credit(5 pts)An unbalanced dataset (e.g. 95% vs 5%) can be problematic even in the training phase. The learned function canbe trivial, e.g. always predicting one class.A possible solution can have a weight for each point in the way that making  a  mistake  in  the  minority  class  will  countmore w.r.t.  the  other.  Please  redefine  the  likelihood  of  the logistic regression to consider these weights for each point. Please compute the log-likelihood and its derivatives.In addition, add to the negative log-likelihood the norm of W (sum of the square ofeach component) and compute the  derivatives.You  should  not  expect  class_weight  parameters  and  SMOTE  to  give  the  exact  same  results because they are different methods.

Ans: Logistic regression is used for estimating probabilities. It's for binary classification problems. After having input parameters, the model will calculate the probability of each observation belonging to one of the two classes. Imbalanced datasets pose challenges to ordinary classification algorithms such as Logistic Regression (LR) and Support Vector Machines (SVM). Our goal is to infer the parameter set vect_(w) = {w0, . . . , w2n} that maximizes the likelihood function. We denote the log-likelihood function as

![picture](https://drive.google.com/uc?export=view&id=1gN_3lY4cInPyadM6eg6LJZG4RHJDcQEU)

Let, we have a dataset D and the parameter 'w' and define likelihood Probability density function p(D|w) given in eq(1).

![picture](https://drive.google.com/uc?export=view&id=1gVWDLWIMnT0FMK8MVRVtcvOcGrxxEngv)


![picture](https://drive.google.com/uc?export=view&id=1gS2cOmguiM029h9hUlJraXZbuDJXiaYY)

To maximize likelihood, usually we minimize the negative “log-likelihood” (NLL).“Log-likelihood” is short for “logarithm of the likelihood”.